In [1]:
!pip install pandas groq tiktoken getpass


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


In [2]:
import getpass
from groq import Groq
import time

In [4]:
groq_api_key = getpass.getpass(prompt="Insira a tua GROQ API_KEY: ")


Insira a tua GROQ API_KEY: ········


In [5]:
client = Groq(api_key=groq_api_key)
# Modelo LLaMA 4 Scout Instruct
model = "meta-llama/llama-4-scout-17b-16e-instruct"

total_tokens_used = 0

def run_llama_prompt(prompt: str) -> str:
    global total_tokens_used
    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=512
    )
    usage = getattr(completion, 'usage', None)
    if usage and hasattr(usage, 'total_tokens'):
        total_tokens_used += usage.total_tokens
    return completion.choices[0].message.content.strip()

# Score validation
def validate_score(score: str) -> float:
    try:
        val = float(score)
    except:
        return 0.0
    if val < -5:
        return -5.0
    if val > 10:
        return 10.0
    return val

# Scoring functions
def score_skills(skills: str, job_desc: str) -> float:
    prompt = (
        f"Given the candidate's skills: {skills}\n"
        f"And the job description: {job_desc}\n"
        "Rate on -5 to 10. Respond with just the number."
    )
    return validate_score(run_llama_prompt(prompt))


def score_education(education: str, job_desc: str) -> float:
    prompt = (
        f"Given the candidate's education: {education}\n"
        "Award extra points for high GPA (no penalties for low).\n"
        f"Job desc: {job_desc}\n"
        "Rate on -5 to 10. Respond with just the number."
    )
    return validate_score(run_llama_prompt(prompt))


def score_experience(experience: str, job_desc: str) -> float:
    prompt = (
        f"Given the candidate's experience: {experience}\n"
        f"And the job description: {job_desc}\n"
        "Rate on -5 to 10. Respond with just the number."
    )
    return validate_score(run_llama_prompt(prompt))


def score_summary(summary: str, job_desc: str) -> float:
    prompt = (
        f"Given the candidate's summary: {summary}\n"
        f"And the job description: {job_desc}\n"
        "Rate on -5 to 10. Respond with just the number."
    )
    return validate_score(run_llama_prompt(prompt))



In [6]:
import glob, json, pandas as pd

# Load job descriptions (JobDescription1.json ... JobDescription5.json)
job_files = sorted(glob.glob('JobDescription*.json'))
jobs = []
for jf in job_files:
    with open(jf, 'r') as f:
        jobs.append(json.load(f))

# Load CVs (cv_1.json ... cv_10.json)
cv_files = sorted(glob.glob('cv_*.json'))
profiles = []
for cf in cv_files:
    with open(cf, 'r') as f:
        profiles.append(json.load(f))

# Score agregations
results = []
for profile in profiles:
    for job in jobs:
        # Concatenate job text fields
        job_text = (
            job.get('about_the_role', '') + ' ' +
            job.get('key_responsibilities', '') + ' ' +
            job.get('requirements', '')
        )
        # Use job-specific weights
        w = job.get('weights', {"summary":1, "experience":1, "education":1, "skills":1})
        total_w = sum(w.values()) if sum(w.values()) != 0 else 1
        # Compute individual scores
        s_sk = score_skills(profile.get('skills', ''), job_text)
        s_ed = score_education(profile.get('education', ''), job_text)
        s_ex = score_experience(profile.get('experience', ''), job_text)
        s_sm = score_summary(profile.get('summary', ''), job_text)
        # Weighted final score
        final = (
            w.get('skills',0)*s_sk +
            w.get('education',0)*s_ed +
            w.get('experience',0)*s_ex +
            w.get('summary',0)*s_sm
        ) / total_w
        results.append({
            'candidate_name': profile.get('name', ''),
            'job_title': job.get('title', ''),
            'final_score': final
        })

# 5. Exibir resultados
results_df = pd.DataFrame(results)
print("Final Scores:")
print(results_df)
print(f"Total tokens used by Groq API: {total_tokens_used}")

# 6. Exportar para CSV, se desejado
results_df.to_csv('profiles_scored.csv', index=False)
print("Results saved to profiles_scored.csv")


Resultados de exemplo:
                                             summary  \
0  Data analyst with financial modeling and visua...   
1  Backend engineer with microservices architectu...   

                                           education  \
0  University of Oxford (Ranked 5), MSc in Financ...   
1  Stanford University (Ranked 2), BSc in Compute...   

                                          experience  \
0  4 years at FinAnalytics LLC focusing on dashbo...   
1  3 years at CloudNative Inc building scalable A...   

                                  skills  final_score_job_1  \
0         Python; SQL; Tableau; Power BI               6.25   
1  Java; Spring Boot; Docker; Kubernetes              -3.25   

   final_score_job_2  final_score_job_3  final_score_job_4  final_score_job_5  
0               6.25               4.75               6.25               6.25  
1              -3.25              -3.25              -3.25              -3.25  
Total de tokens usados pela Groq API: 35

AttributeError: 'dict' object has no attribute 'loc'